In [1]:
# Develop a different way to assess a model based on the Benter paper
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m

from helper import *

# Get the data
main = pd.read_pickle("../Data/main_1.df")

In [2]:
# Log liklihood calculator
# multiply together the probabilities that the winning horse was given across the associated races

def L(preds, data, a=1, b=1):
    # Calculate the LL 
    data = data.copy()
    races = np.unique(data["race_id"])
    
    LLmodel = 0.
    for r in races:
        
        race = data.loc[data["race_id"]==r] 
        raceI = race.index
        raceSize = race.shape[0]
        
        winnerIndex = race.loc[race["won"]==1].index
        if winnerIndex.shape[0]!=1:
            continue
            
        modelPreds = preds[winnerIndex].values
        
        # Get the relevant coefficents
        modelSum = 0.
        
        odds = race["win_odds"]
        probs = [1./o for o in odds]
        normProbs = [p/sum(probs) for p in probs]
        for i in range(raceSize):
            # Get odds probs
            index = raceI[i]
            oddsProbability = normProbs[i]
            if index==winnerIndex:
                winningProb = oddsProbability
                
            modelSum += m.exp(a*m.log(preds[index]) + b*m.log(oddsProbability))
            
        # Now get the numerator
        modelNumer = m.exp(a*m.log(modelPreds) + b*m.log(winningProb))
        c = modelNumer/modelSum
        
        LLmodel += m.log(c)
        
    return LLmodel
        
def RSq(preds, data, a=1, b=1):
    data = data.copy()
    
    # Get random preds
    randPreds = getRandPreds(data)
    
    LLmodel = L(preds, data, a, b)
    LLbl = L(randPreds, data, a, b)
    
    return 1 - LLmodel/LLbl

def oddsProbs(testing):
    testing = testing.copy()
    races = np.unique(testing["race_id"])
    testing["probs"] = 0.
    for r in races:
        race = testing.loc[testing["race_id"]==r]
        raceI = race.index
        raceSize = race.shape[0]
        odds = race["win_odds"]
        oddsProbs = [1./i for i in odds.values]
        normOddsProbs = [i/sum(oddsProbs) for i in oddsProbs]
        
        for i in range(raceSize):
            ind = raceI[i]
            p = normOddsProbs[i]
            testing.loc[ind, "probs"] = p
    
    return testing["probs"]
    
def getRandPreds(data):
    data = data.copy()
    
    # Get LL of random preds
    randomProbs = np.ones(len(data))
    races = np.unique(data["race_id"])
    for r in races:
        race = data.loc[data["race_id"]==r]
        raceI = race.index
        raceSize = race.shape[0]
        if raceSize!=0:
            randomPreds = 1./raceSize
            randomProbs[raceI] = randomPreds
            
    return pd.Series(randomProbs)

In [6]:
oddsPreds = oddsProbs(main)
Rsq = RSq(oddsPreds, main, a=1,b=0)
print("RSquared of betting best odds: {}".format(Rsq))

-12984.746617687586
-15962.042002695001
RSquared of betting best odds: 0.18652346513715068


In [4]:
# Next step is to find LL of a given model and maximize it using a and b# SCREW THIS MOVE ON TO ACTUAL SIMULATINOS